In [1]:
# import topopy
from test_functions import *
# import matplotlib.pyplot as plt
# import samplers
# import seaborn as sns
# from matplotlib import patches, collections
import numpy as np
from ipywidgets import interact, HTML
import random


In [2]:
from sidepanel import SidePanel
import ipywidgets as widgets

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget
from ipyregulus.filters.filters import * 
from ipyregulus.filters.monitor import Monitor
from ipyregulus.tree.has_tree import *
from ipyregulus.alg.view import *

In [3]:
test_function = gerber_different_heights
samples = 50
X = np.random.rand(samples,2)
Y = test_function(X)

# Y = (Y - np.min(Y)) / (np.max(Y) - np.min(Y)) # Not sure whether this is needed 

all_pts = np.hstack((X,np.expand_dims(Y, axis=1)))

# X = samplers.CVTSampler.generate_samples(samples, 2, 1)


# msc = topopy.MorseSmaleComplex(graph="beta skeleton", gradient="steepest", max_neighbors=8, beta=1.0)
# msc.build(X, Y)


# NEW PART IN REG to create pts at runtime 

In [4]:
from regulus.topo import morse_smale, Regulus
from regulus.data.data import Data
from regulus.measures import *
from regulus.models import *
import pandas as pd

cols = ['x'+str(i) for i in range(all_pts.shape[1])]

cols[-1] = 'y'

df = pd.DataFrame(all_pts, columns=cols)

pts = Data.from_pts(df)

# Normalize here  
pts.normalize()

regulus = morse_smale(pts, knn=8)#**kwargs)

regulus.add_attr('linear', linear_model)
regulus.add_attr('fitness', fitness)
regulus.add_attr('relative_fitness', relative_fitness)

regulus.tree.add_attr('parent_fitness', parent_fitness)
regulus.tree.add_attr('child_fitness', child_fitness)
regulus.tree.add_attr('size', node_size)
regulus.tree.add_attr('rel_size', node_relative_size)
regulus.tree.add_attr('span', node_span)




found 3 singles
len(idx)= 48


/media/orpheus92/OS/conda/envs/regulus/lib/python3.6/site-packages/numpy/core/fromnumeric.py:51: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)
/media/orpheus92/OS/conda/envs/regulus/lib/python3.6/site-packages/numpy/core/fromnumeric.py:51: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


In [5]:
from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget

tree = TreeWidget(regulus.tree)
data = DataWidget(data=regulus)


# TreeView

In [6]:
v2 = show_tree(tree)

w = widgets.Dropdown(
    options=['span', 'fitness', 'parent_fitness', 'child_fitness'],
    value='span',
    description='Attribute:',
    disabled=False,
)
panel = SidePanel(title='Fitness')#, side='split-bottom', ref='SidePanel')
with panel:
    display(w, v2.view, v2.filter)
widgets.link((w, 'value'), (v2.view, 'field'))

TreeView(field='span', tree_model=TreeWidget(attrs={'span': {0: 0.7029088917768025, 1: 0.14566526902549265, 2:…

# Details View

In [7]:
details = DetailsView(data=data)

p7 = SidePanel(title='details')
with p7:
    display(details)

# Linking Two Views

In [8]:

widgets.link((v2.view, 'details'), (details, 'show'))


# Create resample class, resampling a user specified partition

In [9]:
from resample import get_pts, Resample

# Change the following line to use differennt partition to resample
ind = get_pts(data.partitions[1],data.pts_loc)

selectP = df.loc[ind,:]
r = Resample(selectP)

# A line chart for a specified partition, different interactions could be added 

In [10]:
from cyplot import plot
myp = plot(selectP,index = 'y',debug=True)
myp.enable('brush_x')
myp.on('brush_x', r.cb)
display(myp)

# random generate pts based on user selected range of function value

In [11]:
r.add_samples(10) # user's specified # of samples
# r.vals 



# When user adds enough samples, they could use regression to predict the point locations to resample 

In [12]:
# Would scale back if needed 
X2 = r.get_input()

new_x = np.vstack((X,X2))
new_y = test_function(new_x)

In [13]:
new_y.shape

(60,)